<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-16 16:21:14
-------------------
qualified stocks: 95
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  2.11 L
Current:  1.28 C
-------------------
Today PnL: 1.43 L (1.13%)
Current PnL: -15.79 L (-11.2%)
CY Booked + Current PnL: -12.67 L (-8.98%)
-------------------
Total profit:  4.32 L
Total loss:  -20.12 L
-------------------
Total Booked + Current PnL: 14.31 L (12.55%)
Total Booked PnL: 30.11 L (26.4%)
Curr Year Booked PnL: 3.13 L (2.44%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 78.50 L (61.29%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.40 C (109.29%)
Deployed:  1.14 C
Current:  1.28 C
CAGR/XIRR %: 8.39%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RELIANCE,1288.53,1526.00,6.87,X-LC,91.58,230111.0,26523.0,10999.0,0.15,13.03,4.78,18.43,157.0,2.41,1.83,25.35,XY25,NTT,REFINERIES
69,SBILIFE,1496.49,1924.99,0.20,H-LC,76.84,148966.0,23261.0,12737.0,-0.63,18.50,8.55,28.63,147.0,1.83,1.18,32.71,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-16.34,X-LC,32.63,92736.0,17614.0,14541.0,-0.65,23.45,15.68,42.81,120.0,1.21,0.74,16.30,X40,BTT,CHEMICALS
6,ANGELONE,2328.67,3033.00,11.80,H-SC,95.79,182039.0,30675.0,15109.0,9.03,20.27,8.30,30.25,111.0,2.03,1.44,44.08,X40N,NTT,FINANCE
70,SBIN,760.30,863.00,-14.39,M-LC,42.11,209906.0,8426.0,18787.0,0.03,4.18,8.95,13.51,60.0,0.45,1.67,17.31,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TITAGARH,1097.23,1548.0,-9.03,H-SC,86.32,159749.0,-32266.0,111153.0,12.83,-16.80,69.58,41.08,95.0,-0.29,1.27,35.21,XY24,NTT,ENGINEERING
6,ANGELONE,2328.67,3033.0,11.80,H-SC,95.79,182039.0,30675.0,15109.0,9.03,20.27,8.30,30.25,111.0,2.03,1.44,44.08,X40N,NTT,FINANCE
89,VALIANTORG,512.64,838.0,-328.95,H-SC,57.89,97422.0,-35864.0,120462.0,9.00,-26.91,123.65,63.47,161.0,-0.30,0.77,63.30,XR,NTT,CHEMICALS
51,LUXIND,1448.19,2442.0,43.16,M-SC,66.32,148608.0,11030.0,83384.0,7.52,8.02,56.11,68.62,79.0,0.13,1.18,28.83,X40N,NTT,TEXTILES
76,STARHEALTH,551.56,761.0,0.02,M-SC,89.47,163677.0,-36539.0,112561.0,7.33,-18.25,68.77,37.97,62.0,-0.32,1.30,31.42,XY24,NTT,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1022.60,1486.00,3.67,X-MC,2.11,182567.0,27132.0,43305.0,-3.39,17.46,23.72,45.32,201.0,0.63,1.45,24.22,XY24,NTT,HEALTHCARE
86,UJJIVANSFB,52.77,60.00,38.86,M-SC,78.95,120393.0,-22086.0,41608.0,-1.81,-15.50,34.56,13.70,214.0,-0.53,0.96,42.10,X40N,NTT,BANKS
20,COFFEEDAY,59.14,80.00,-61.77,L-SC,35.79,55872.0,-57677.0,97726.0,-1.79,-50.79,174.91,35.27,166.0,-0.59,0.44,36.75,XR,NTT,HOTELS
78,SYMPHONY,1311.84,2013.49,-21.02,M-SC,74.74,147948.0,-2914.0,83605.0,-1.70,-1.93,56.51,53.49,15.0,-0.03,1.17,30.18,X40N,ATH,DURABLES
41,INFY,1432.81,2275.00,-11.34,X-LC,73.68,220996.0,21835.0,95227.0,-1.43,10.96,43.09,58.78,10.0,0.23,1.75,15.38,X40,BTT,IT


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TATAMOTORS,776.02,1065.00,-51.41,M-LC,94.74,193636.0,-12009.0,88588.0,0.36,-5.84,45.75,37.24,1.0,-0.14,1.54,26.04,XY24,NTT,AUTO
38,INDIAMART,2311.97,4911.36,-52.21,H-SC,67.37,103409.0,1682.0,112695.0,0.85,1.65,108.98,112.43,2.0,0.01,0.82,22.05,AR,ATH,MISC
2,ABB,5319.37,8953.12,-36.37,H-LC,62.11,145500.0,12516.0,78323.0,1.48,9.41,53.83,68.31,4.0,0.16,1.15,18.64,AR,ATH,ELECTRICAL
82,TCS,3928.29,4998.00,-14.87,X-LC,61.05,192310.0,-19818.0,77578.0,-0.52,-9.34,40.34,27.23,5.0,-0.26,1.53,10.21,X40,BTT,IT
49,LICI,777.74,983.00,-13.28,H-LC,53.68,191968.0,17754.0,28219.0,1.99,10.19,14.70,26.39,6.0,0.63,1.52,17.53,XY25,NTT,INSURANCE


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DMART,3484.00,5112.00,-18.32,X-LC,17.89,100553.0,16937.0,22132.0,2.85,20.26,22.01,46.73,146.0,0.77,0.80,23.08,AR,NTT,FMCG
56,PIDILITIND,2504.06,3576.00,-16.34,X-LC,32.63,92736.0,17614.0,14541.0,-0.65,23.45,15.68,42.81,120.0,1.21,0.74,16.30,X40,BTT,CHEMICALS
6,ANGELONE,2328.67,3033.00,11.80,H-SC,95.79,182039.0,30675.0,15109.0,9.03,20.27,8.30,30.25,111.0,2.03,1.44,44.08,X40N,NTT,FINANCE
57,POLYCAB,5005.92,7541.35,9.13,H-MC,83.16,170646.0,30480.0,40511.0,0.87,21.75,23.74,50.65,55.0,0.75,1.35,30.24,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-0.40,M-LC,80.00,192814.0,41608.0,24063.0,0.09,27.52,12.48,43.43,37.0,1.73,1.53,33.65,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,43.16,M-SC,66.32,148608.0,11030.0,83384.0,7.52,8.02,56.11,68.62,79.0,0.13,1.18,28.83,X40N,NTT,TEXTILES
6,ANGELONE,2328.67,3033.00,11.80,H-SC,95.79,182039.0,30675.0,15109.0,9.03,20.27,8.30,30.25,111.0,2.03,1.44,44.08,X40N,NTT,FINANCE
57,POLYCAB,5005.92,7541.35,9.13,H-MC,83.16,170646.0,30480.0,40511.0,0.87,21.75,23.74,50.65,55.0,0.75,1.35,30.24,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-0.40,M-LC,80.00,192814.0,41608.0,24063.0,0.09,27.52,12.48,43.43,37.0,1.73,1.53,33.65,XY24,NTT,MISC
33,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,82.11,152086.0,1173.0,59283.0,0.47,0.78,38.98,40.06,14.0,0.02,1.21,24.40,AR,ATH,AUTO


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,-3.11,H-SC,20.00,108945.0,6941.0,38087.0,4.41,6.80,34.96,44.14,22.0,0.18,0.86,24.15,XR,ATH,MISC
94,WIPRO,243.46,420.00,-7.89,M-LC,34.74,157672.0,6727.0,102723.0,-0.88,4.46,65.15,72.51,52.0,0.07,1.25,18.71,XR,NTT,IT
33,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,82.11,152086.0,1173.0,59283.0,0.47,0.78,38.98,40.06,14.0,0.02,1.21,24.40,AR,ATH,AUTO
12,BANKINDIA,113.49,190.00,-33.09,M-MC,13.68,130108.0,956.0,86118.0,0.25,0.74,66.19,67.42,41.0,0.01,1.03,26.54,XR,NTT,BANKS
38,INDIAMART,2311.97,4911.36,-52.21,H-SC,67.37,103409.0,1682.0,112695.0,0.85,1.65,108.98,112.43,2.0,0.01,0.82,22.05,AR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VALIANTORG,512.64,838.00,-328.95,H-SC,57.89,97422.0,-35864.0,120462.0,9.00,-26.91,123.65,63.47,161.0,-0.30,0.77,63.30,XR,NTT,CHEMICALS
86,UJJIVANSFB,52.77,60.00,38.86,M-SC,78.95,120393.0,-22086.0,41608.0,-1.81,-15.50,34.56,13.70,214.0,-0.53,0.96,42.10,X40N,NTT,BANKS
12,BANKINDIA,113.49,190.00,-33.09,M-MC,13.68,130108.0,956.0,86118.0,0.25,0.74,66.19,67.42,41.0,0.01,1.03,26.54,XR,NTT,BANKS
69,SBILIFE,1496.49,1924.99,0.20,H-LC,76.84,148966.0,23261.0,12737.0,-0.63,18.50,8.55,28.63,147.0,1.83,1.18,32.71,AR,ATH,INSURANCE
87,UNIONBANK,123.87,163.00,-14.11,M-LC,38.95,151244.0,10404.0,34090.0,0.64,7.39,22.54,31.59,48.0,0.31,1.20,31.38,XY24,NTT,BANKS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VIPIND,488.80,718.00,-880.60,H-SC,100.00,74344.0,-20972.0,65668.0,4.22,-22.00,88.33,46.89,82.0,-0.32,0.59,46.63,X40N,NTT,MISC
80,TATAELXSI,7332.28,9000.00,-17.16,X-MC,98.95,86968.0,-15684.0,39031.0,0.51,-15.28,44.88,22.74,47.0,-0.40,0.69,30.72,X40N,NTT,IT
75,SONACOMS,578.05,1006.00,-26.58,M-MC,97.89,94999.0,-6160.0,81053.0,1.71,-6.09,85.32,74.03,49.0,-0.08,0.75,33.51,AR,BTT,AUTO
79,TANLA,917.30,1963.11,-39.00,H-SC,96.84,149175.0,-90240.0,363196.0,2.71,-37.69,243.47,114.01,20.0,-0.25,1.18,36.82,AR,ATH,IT
6,ANGELONE,2328.67,3033.00,11.80,H-SC,95.79,182039.0,30675.0,15109.0,9.03,20.27,8.30,30.25,111.0,2.03,1.44,44.08,X40N,NTT,FINANCE


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.28
1,25,38.95
2,50,68.28


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.99
X40N    15.45
X40     14.38
AR      14.11
XR      10.39
XY25     8.35
X200     1.20
SR       1.11
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.74
M-SC    17.93
X-LC    16.26
H-MC     9.01
M-LC     8.36
H-LC     5.93
X-MC     5.61
M-MC     5.57
X-SC     2.81
L-SC     2.28
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.75,-0.31,32.22
BANKS,10.97,-9.08,62.58
IT,9.21,-19.08,85.51
FINANCE,7.57,-15.99,68.33
MISC,7.20,-24.72,78.38
INSURANCE,6.19,1.10,28.72
PAINTS,5.61,-16.29,50.71
AUTO,4.42,-8.79,56.21
ELECTRICAL,4.13,-6.61,60.94


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2943475.0,27
AR,1285047.0,15
XR,1235530.0,15
X40N,1214714.0,18
X40,783139.0,12
SR,203945.0,2
XY25,118189.0,5
X200,66434.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2322648.0,22
M-SC,1980239.0,21
X-LC,694306.0,12
M-MC,564986.0,5
H-MC,471297.0,8
X-SC,353066.0,4
M-LC,326989.0,6
L-SC,310801.0,4
X-MC,298102.0,5


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      893557.0      6
M-SC       XY24      745735.0      6
H-SC       AR        535830.0      3
           X40N      519265.0      7
M-SC       X40N      441273.0      6
           XR        422420.0      5
M-MC       XY24      397815.0      3
X-LC       X40       374261.0      5
H-SC       XR        285691.0      4
M-LC       XY24      205479.0      4
H-LC       AR        196234.0      4
M-SC       X40       193988.0      2
X-LC       AR        182428.0      3
H-MC       XY24      169253.0      2
L-SC       XR        163209.0      2
X-MC       XY24      144060.0      2
H-MC       AR        132739.0      2
X-SC       SR        120283.0      1
           XR        117241.0      1
           X40N      115542.0      2
M-LC       XR        102723.0      1
X-MC       X40N       98123.0      2
M-SC       AR         93161.0      1
H-SC       X40        88305.0      2
M-MC       XR         86118.0      1
L-SC       XY24       83990.0      1
M-SC       SR         83662.0      1
M-MC       AR         81053.0      1
X-LC       XY25       71183.0      3
H-MC       X40        70666.0      2
X-LC       X200       66434.0      1
L-SC       AR         63602.0      1
H-MC       XR         58128.0      1
X-MC       X40        55919.0      1
H-MC       X40N       40511.0      1
H-LC       XY25       28219.0      1
M-LC       XY25       18787.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
